# Sistema de Transcripción y Análisis de Audios

Este notebook permite ejecutar los agentes de transcripción y análisis de calidad, y visualizar las métricas generadas.

## Contenido
1. Configuración inicial
2. Agente de Transcripción
3. Visualización de Métricas de Transcripción
4. Agente de Auditoría de Calidad de Servicio al Cliente
5. Visualización de Métricas de Calidad
6. Reportes Consolidados

## 1. Configuración Inicial

In [1]:
import logging
import sys

# 1. Obtenemos el logger raíz
logger = logging.getLogger()

# 2. Eliminamos configuraciones previas que puedan estar bloqueando la salida
for handler in logger.handlers[:]:
    logger.removeHandler(handler)

# 3. Configuramos la nueva salida hacia stdout (la consola del notebook)
logging.basicConfig(
    level=logging.INFO, # Cambia a logging.DEBUG si quieres ver aún más detalle
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout)
    ],
    force=True # Forzamos la configuración incluso si otra librería la cambió
)

logging.info("✓ Sistema de logs configurado para Notebook")

2026-01-10 05:55:45,549 - INFO - ✓ Sistema de logs configurado para Notebook


In [4]:
# Importar librerías necesarias
import sys
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Importar módulos del proyecto
import config
from agents.transcription_agent import TranscriptionAgent
from agents.greeting_agent import GreetingAgent, QualityAgent
from agents.audio_processor import AudioProcessor

print("✓ Librerías importadas correctamente")
print(f"✓ Directorio de audios: {config.AUDIO_DIR}")
print(f"✓ Directorio de salida: {config.OUTPUT_DIR}")

✓ Librerías importadas correctamente
✓ Directorio de audios: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\data\audios\MOVIL
✓ Directorio de salida: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output


## 2. Agente de Transcripción

Este agente:
- Elimina silencios y tonos de espera
- Transcribe audios a texto usando Whisper
- Identifica participantes (speaker diarization)
- Genera métricas de calidad

In [4]:
# Verificar archivos de audio disponibles
audio_extensions = ['.wav', '.mp3', '.m4a', '.flac', '.ogg']
audio_files = []
for ext in audio_extensions:
    audio_files.extend(config.AUDIO_DIR.glob(f"*{ext}"))

print(f"Archivos de audio encontrados: {len(audio_files)}")
for i, audio_file in enumerate(audio_files, 1):
    print(f"  {i}. {audio_file.name}")

if not audio_files:
    print("\n ADVERTENCIA: No se encontraron archivos de audio.")
    print(f"   Por favor, coloca archivos de audio en: {config.AUDIO_DIR}")

Archivos de audio encontrados: 50
  1. 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav
  2. 9155033327270001731 999799232 75316202 MASIVO_MOVIL ABAI 0906.wav
  3. 9155034794510001731 945206356 09337147 MASIVO_MOVIL GSS 0906 _1.wav
  4. 9155036654930001731 945206356 72467112 MASIVO_MOVIL KONECTA 0906.wav
  5. 9155041036900001731 964061323 72396494 MASIVO_MOVIL ABAI 1006 _1.wav
  6. 9155041370430001731 964061323 48576957 MASIVO_MOVIL ABAI 1006.wav
  7. 9155042319270001731 998811016 74812275 MASIVO_MOVIL KONECTA 1006 _1.wav
  8. 9155042371100001731 998811016 41398045 MASIVO_MOVIL KONECTA 1006.wav
  9. 9155042433790001731 945668019 43191453 MASIVO_MOVIL KONECTA 1006 _1.wav
  10. 9155042582490001731 945668019 70088919 MASIVO_MOVIL GSS 1006.wav
  11. 9155050433020001731 924042582 77178070 MASIVO_MOVIL KONECTA 1106 _1.wav
  12. 9155050514900001731 924042582 77038601 MASIVO_MOVIL KONECTA 1106.wav
  13. 9155050950490001731 985994278 46042916 MASIVO_MOVIL GSS 1106 _1.wav
  1

In [6]:
# Verificar archivos de transcripción existentes
transcription_files = list(config.TRANSCRIPTION_DIR.glob("*_transcription.json"))

print(f" Archivos de transcripción encontrados: {len(transcription_files)}")

if transcription_files:
    print(f"\n✓ Ya existen {len(transcription_files)} transcripciones.")
    print(f"  Puedes saltar la transcripción y pasar directamente al análisis de calidad.")
    print(f"\n  Para re-transcribir, ejecuta las siguientes celdas.")
else:
    print(f"\n No hay transcripciones. Debes ejecutar el agente de transcripción primero.")

 Archivos de transcripción encontrados: 0

 No hay transcripciones. Debes ejecutar el agente de transcripción primero.


In [7]:
# Inicializar agente de transcripción
print("Inicializando agente de transcripción...")
transcription_agent = TranscriptionAgent(
    whisper_model_name=config.WHISPER_MODEL,
    use_gpu=True
)
print("✓ Agente de transcripción listo")

Inicializando agente de transcripción...
2026-01-10 05:56:34,399 - INFO - Cargando Whisper 'large-v3-turbo' en cuda...
2026-01-10 05:56:34,845 - INFO - HTTP Request: GET https://huggingface.co/api/models/mobiuslabsgmbh/faster-whisper-large-v3-turbo/revision/main "HTTP/1.1 307 Temporary Redirect"
2026-01-10 05:56:34,976 - INFO - HTTP Request: GET https://huggingface.co/api/models/dropbox-dash/faster-whisper-large-v3-turbo/revision/main "HTTP/1.1 200 OK"
2026-01-10 05:56:39,555 - INFO - HTTP Request: HEAD https://huggingface.co/pyannote/speaker-diarization-3.1/resolve/main/config.yaml "HTTP/1.1 200 OK"
2026-01-10 05:56:41,002 - DEBUG - Registered checkpoint save hook for _speechbrain_save
2026-01-10 05:56:41,003 - DEBUG - Registered checkpoint load hook for _speechbrain_load
2026-01-10 05:56:41,003 - DEBUG - Registered checkpoint save hook for save
2026-01-10 05:56:41,003 - DEBUG - Registered checkpoint load hook for load
2026-01-10 05:56:41,276 - INFO - Applied quirks (see `speechbrain.

### 3.1 Procesar Audios

Tienes dos opciones para procesar los audios:
1. **Opción A (Solo Transcripción)**: Mucho más rápida, ideal si solo necesitas el texto.
2. **Opción B (Solo Diarización)**: Añade la identificación de quién habla a los resultados de la Opción A. Es más lenta y requiere GPU.

In [8]:
# OPCIÓN A: Solo Transcripción (Rápido)
print("Iniciando SOLO TRANSCRIPCIÓN...\n")
transcription_results = transcription_agent.process_directory(
    audio_dir=str(config.AUDIO_DIR),
    output_dir=str(config.TRANSCRIPTION_DIR),
    preprocess=True,
    diarize=False  #Desactivamos la diarización para mayor velocidad
)

print(f"\n Transcripciones completadas: {len(transcription_results)}")

Iniciando SOLO TRANSCRIPCIÓN...

Encontrados 50 archivos de audio

Procesando 1/50: 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav
2026-01-10 05:56:45,728 - INFO - Preprocesando audio...
Audio cargado exitosamente: 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav (422.9s, 8000Hz)
2026-01-10 05:56:46,383 - INFO - Aplicando filtro paso banda...
2026-01-10 05:56:50,716 - INFO - Eliminando tonos de espera...
2026-01-10 05:56:53,074 - INFO - Eliminando silencios...
2026-01-10 05:56:56,167 - INFO - Exportando para Whisper...
2026-01-10 05:56:56,200 - INFO - Audio exportado exitosamente con pydub: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\audioPro\temp_9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav
2026-01-10 05:56:56,201 - INFO - Iniciando transcripción: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\audioPro\temp_9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav
Audio cargado exitosamente: temp_9155033267980001731

In [9]:
# OPCIÓN B: Solo Diarización (Diferida)
print("Iniciando SOLO DIARIZACIÓN basada en transcripción previa...\n")
if 'transcription_results' in locals() and transcription_results:
    transcription_results = transcription_agent.add_diarization_to_results(
        results=transcription_results
    )
    print(f"\n Diarización completada para {len(transcription_results)} audios")
else:
    print("Error: No se encontraron resultados de transcripción. Por favor, ejecuta la OPCIÓN A primero.")

Iniciando SOLO DIARIZACIÓN basada en transcripción previa...

Usando audio procesado encontrado en audioPro: temp_9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav

--- Iniciando diarización para: 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav ---
2026-01-10 06:24:23,063 - INFO - Diarización completada en 170.01s
Transcripción guardada en: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\transcriptions\9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1_transcription.json
✓ Diarización completada para 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1.wav
Usando audio procesado encontrado en audioPro: temp_9155033327270001731 999799232 75316202 MASIVO_MOVIL ABAI 0906.wav

--- Iniciando diarización para: 9155033327270001731 999799232 75316202 MASIVO_MOVIL ABAI 0906.wav ---
2026-01-10 06:26:12,479 - INFO - Diarización completada en 109.41s
Transcripción guardada en: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\transcriptions\91550

## 📈 Visualización de Métricas de Transcripción (Audio → Texto)

Esta sección muestra métricas del proceso de transcripción:
- Calidad de transcripción (quality scores)
- Confianza del modelo Whisper
- Eficiencia de procesamiento
- Efectividad del preprocesamiento
- Análisis de speakers

In [2]:
# Importar módulo de visualización
from visualize_transcription_metrics import generate_all_visualizations

print("📊 Generando visualizaciones de métricas de transcripción...\n")

# Generar todas las visualizaciones
figures, df_trans = generate_all_visualizations(str(config.TRANSCRIPTION_DIR))

📊 Generando visualizaciones de métricas de transcripción...

Cargando metricas de 50 transcripciones...
Cargadas 50 transcripciones exitosamente

Generando visualizaciones interactivas...
7 visualizaciones generadas


### 📊 Dashboard Resumen

In [4]:
# Mostrar dashboard con KPIs
if 'dashboard' in figures:
    figures['dashboard'].show()
else:
    print("⚠️ No hay datos de transcripción para visualizar")

### 📊 Distribución de Calidad

In [5]:
# Distribución de quality scores
if 'quality_distribution' in figures:
    figures['quality_distribution'].show()

### 🎯 Confianza del Modelo

In [6]:
# Scores de confianza por archivo
if 'confidence_scores' in figures:
    figures['confidence_scores'].show()

### 📈 Duración vs Calidad

In [7]:
# Relación entre duración y calidad
if 'duration_vs_quality' in figures:
    figures['duration_vs_quality'].show()

### ⚡ Eficiencia de Procesamiento

In [8]:
# Tiempo de procesamiento y ratio de tiempo real
if 'processing_efficiency' in figures:
    figures['processing_efficiency'].show()

### 🔧 Efectividad del Preprocesamiento

In [3]:
# Métricas de preprocesamiento: silencio removido, tonos de espera, etc.
if 'preprocessing_effectiveness' in figures:
    figures['preprocessing_effectiveness'].show()

### 👥 Análisis de Speakers

In [17]:
# Distribución y tiempo por speaker
if 'speaker_analysis' in figures:
    figures['speaker_analysis'].show()

### 📋 Tabla de Datos

In [5]:
# Mostrar tabla con métricas
if not df_trans.empty:
    print("\n📊 Métricas de Transcripción:\n")
    display(df_trans[[
        'archivo', 'quality_score', 'avg_confidence', 
        'audio_duration', 'processing_time', 'num_speakers'
    ]].head(10))
    
    # Exportar a CSV
    trans_report_path = config.OUTPUT_DIR / "reporte_transcripcion_metricas.csv"
    df_trans.to_csv(trans_report_path, index=False, encoding='utf-8-sig')
    print(f"\n✅ Reporte de transcripción exportado a: {trans_report_path}")


📊 Métricas de Transcripción:



,archivo,quality_score,avg_confidence,audio_duration,processing_time,num_speakers
0,9155033267980001731 999799232 70942859 MASIVO_...,89.84,1.0,422.94,188.486930,2
1,9155033327270001731 999799232 75316202 MASIVO_...,91.02,1.0,250.86,118.006780,2
2,9155034794510001731 945206356 09337147 MASIVO_...,88.16,1.0,457.32,115.342643,2
3,9155036654930001731 945206356 72467112 MASIVO_...,86.95,1.0,558.54,204.468517,2
4,9155041036900001731 964061323 72396494 MASIVO_...,86.53,1.0,452.97,170.626613,2
5,9155041370430001731 964061323 48576957 MASIVO_...,93.37,1.0,824.43,366.443508,3
6,9155042319270001731 998811016 74812275 MASIVO_...,90.96,1.0,162.60,82.812665,2
7,9155042371100001731 998811016 41398045 MASIVO_...,89.76,1.0,263.46,99.202674,2
8,9155042433790001731 945668019 43191453 MASIVO_...,93.02,1.0,339.09,151.470197,2
9,9155042582490001731 945668019 70088919 MASIVO_...,92.33,1.0,491.94,206.604542,2



✅ Reporte de transcripción exportado a: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\reporte_transcripcion_metricas.csv


## 4. 🤖 Agente de Auditoría de Calidad de Servicio al Cliente

Este agente:
- **Usa el LLM** para analizar si el asesor cumple con las reglas
- Verifica cumplimiento con criterios JSON
- Genera reporte de compliance con scores detallados

### 4.1. Verificación de Conexión con LLM

**IMPORTANTE**: Esta celda verifica que el LLM (LM Studio) esté funcionando correctamente antes de procesar.

In [2]:
# Importar librerías necesarias
import sys
import json
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Importar módulos del proyecto
import config
#from agents.transcription_agent import TranscriptionAgent
from agents.greeting_agent import GreetingAgent, QualityAgent
#from agents.audio_processor import AudioProcessor

print("✓ Librerías importadas correctamente")
print(f"✓ Directorio de transcripciones: {config.TRANSCRIPTION_DIR}")
print(f"✓ Directorio de salida: {config.OUTPUT_DIR}")

✓ Librerías importadas correctamente
✓ Directorio de transcripciones: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\transcriptions
✓ Directorio de salida: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output


In [ ]:
# Verificar configuración del LLM
print("="*60)
print("VERIFICACIÓN DE CONFIGURACIÓN LLM")
print("="*60)
print(f"\n Configuración:")
print(f"  - Base URL: {config.OPENAI_BASE_URL}")
print(f"  - Modelo: {config.GPT_MODEL}")
print(f"  - API Key configurada: {'Sí' if config.OPENAI_API_KEY else 'No'}")

# Probar conexión
from openai import OpenAI

try:
    print(f"\n Probando conexión con LM Studio...", end=" ")
    test_client = OpenAI(
        api_key=config.OPENAI_API_KEY,
        base_url=config.OPENAI_BASE_URL
    )
    
    response = test_client.chat.completions.create(
        model=config.GPT_MODEL,
        messages=[
            {"role": "system", "content": "Responde brevemente."},
            {"role": "user", "content": "Di 'OK' si funciona."}
        ],
        temperature=0.1,
        max_tokens=10
    )
    
    print(" CONEXIÓN EXITOSA")
    print(f"  - Respuesta: {response.choices[0].message.content}")
    if hasattr(response, 'usage'):
        print(f"  - Tokens usados: {response.usage.total_tokens}")
    print(f"\n El LLM está funcionando correctamente. Puedes continuar con el análisis.")
    
except Exception as e:
    print(" ERROR")
    print(f"\n No se pudo conectar con el LLM: {str(e)}")
    print(f"\n Soluciones:")
    print(f"  1. Verifica que LM Studio esté ejecutándose")
    print(f"  2. Confirma que el servidor esté en {config.OPENAI_BASE_URL}")
    print(f"  3. Asegúrate de que el modelo {config.GPT_MODEL} esté cargado")
    print(f"\n El análisis continuará pero usará reglas básicas en lugar del LLM")

In [3]:
# Definir modelos y sus URLs a comparar
MODELS_TO_COMPARE = [
    
    {
        "model": "qwen3-8b", 
        "url": "http://localhost:1234/v1", 
        "name": "Qwen-Local"
    },
    {
        "model": "meta-llama-3-8b-instruct", 
        "url": "http://localhost:1234/v1", 
        "name": "Llama-Local"
    },
    # Puedes agregar más modelos aquí, incluso de APIs remotas
    # {
    #     "model": "gpt-4o", 
    #     "url": "https://api.openai.com/v1", 
    #     "name": "OpenAI-Cloud"
    # }
]

all_results = {}
print(f"✓ {len(MODELS_TO_COMPARE)} configuraciones de modelos listas para comparación")

✓ 2 configuraciones de modelos listas para comparación


In [ ]:
print(f"Iniciando análisis comparativo con {len(MODELS_TO_COMPARE)} configuraciones de LLM")

for config_item in MODELS_TO_COMPARE:
    model = config_item["model"]
    url = config_item["url"]
    display_name = config_item["name"]
    
    print(f"\n{'#'*60}")
    print(f"PROCESANDO CON: {display_name} ({model})")
    print(f"URL: {url}")
    print(f"{'#'*60}")
    
    # Inicializar agente con el modelo y URL específicos
    agent = QualityAgent(
        criteria_path=str(config.CRITERIOS_JSON),
        use_ai=True,
        model_name=model,
        base_url=url
    )
    
    results = agent.process_directory(
        transcription_dir=str(config.TRANSCRIPTION_DIR),
        output_dir=str(config.GREETING_DIR)
    )
    # Guardar resultados usando el nombre amigable o el modelo como clave
    all_results[display_name] = results
sleep(30)
print("\n✓ Procesamiento multi-modelo y multi-URL completado")

Iniciando análisis comparativo con 2 configuraciones de LLM

############################################################
PROCESANDO CON: Qwen-Local (qwen3-8b)
URL: http://localhost:1234/v1
############################################################
Agente configurado con LangChain
   - Modelo: qwen3-8b
   - Base URL: http://localhost:1234/v1
   Probando conexion con LLM... Conexion exitosa (LangChain)
Encontrados 50 archivos de transcripción

Analizando 1/50: 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1_transcription.json

Evaluando 10 reglas en una sola llamada al LLM...
   Llamando al LLM con LangChain (qwen3-8b)... Respuesta recibida
   Analisis completado
Análisis guardado en: d:\Proy\AntiG\MIA\NLP\TrabajoFinal\output\greeting_analysis\9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1_transcription_qwen3-8b_quality_analysis.json
Completado: 9155033267980001731 999799232 70942859 MASIVO_MOVIL ABAI 0906 _1_transcription.json
  - Modelo: qwen3-8b
 

In [ ]:
# Mostrar resumen de análisis
if greeting_results:
    print("\n" + "="*80)
    print("RESUMEN DE ANÁLISIS DE CALIDAD")
    print("="*80)
    
    for i, result in enumerate(greeting_results[:5], 1):  # Mostrar primeros 5
        print(f"\n{i}. {result['audio_file']}")
        print(f"   Score Global: {result['global_score']}/100")
        print(f"   Cumplimiento: {result['compliance']['global_result']}")
        
        rules = result['rule_analysis']
        
        # Mostrar reglas clave
        r1 = rules.get('R1_validacion_datos', {})
        r2 = rules.get('R2_empatia_claridad', {})
        
        print(f"\n   ✓ R1 Validación/Identificación: {'CUMPLE' if r1.get('cumple') else 'NO CUMPLE'} (Score: {r1.get('score', 0)}/100)")
        print(f"   ✓ R2 Empatía/Saludo: {'CUMPLE' if r2.get('cumple') else 'NO CUMPLE'} (Score: {r2.get('score', 0)}/100)")
        
        # Mostrar evidencia del LLM
        if r2.get('evidencia'):
            print(f"   📝 Evidencia LLM: {r2.get('evidencia')[:100]}...")
    
    if len(greeting_results) > 5:
        print(f"\n... y {len(greeting_results) - 5} análisis más.")
else:
    print("⚠️ No hay resultados de análisis")

## 5. 📊 Visualización de Métricas

In [ ]:
# Preparar datos para visualización
if greeting_results:
    greeting_data = []
    for result in greeting_results:
        rules = result['rule_analysis']
        greeting_data.append({
            'Audio': result['audio_file'][:30],  # Truncar nombre
            'Cumplimiento': result['compliance']['global_result'],
            'Puntuación': result['global_score'],
            'R1_Score': rules.get('R1_validacion_datos', {}).get('score', 0),
            'R2_Score': rules.get('R2_empatia_claridad', {}).get('score', 0),
            'R3_Score': rules.get('R3_ofertas_adecuadas', {}).get('score', 0),
            'R4_Score': rules.get('R4_respeto_decision', {}).get('score', 0),
            'R5_Score': rules.get('R5_formalizacion_cierre', {}).get('score', 0)
        })
    
    df_greeting = pd.DataFrame(greeting_data)
    print("✓ Datos preparados para visualización")
    print(f"\n📊 Estadísticas generales:")
    print(f"  - Puntuación promedio: {df_greeting['Puntuación'].mean():.1f}/100")
    print(f"  - Tasa de cumplimiento: {(df_greeting['Cumplimiento'] == 'CUMPLE').sum()}/{len(df_greeting)} ({(df_greeting['Cumplimiento'] == 'CUMPLE').sum()/len(df_greeting)*100:.1f}%)")
else:
    print("⚠️ No hay datos para visualizar")

In [ ]:
# Gráfico: Distribución de Puntuaciones
if greeting_results:
    fig = go.Figure()
    
    colors = ['green' if score >= 80 else 'orange' if score >= 60 else 'red' 
              for score in df_greeting['Puntuación']]
    
    fig.add_trace(go.Bar(
        x=df_greeting['Audio'],
        y=df_greeting['Puntuación'],
        marker_color=colors,
        text=df_greeting['Puntuación'],
        texttemplate='%{text:.0f}',
        textposition='outside'
    ))
    
    fig.add_hline(y=80, line_dash="dash", line_color="green", 
                  annotation_text="Umbral CUMPLE (80)")
    
    fig.update_layout(
        title='Puntuación Global de Calidad por Audio (Analizado con LLM)',
        xaxis_title='Audio',
        yaxis_title='Puntuación',
        height=500,
        showlegend=False
    )
    
    fig.show()

In [ ]:
# Gráfico: Scores por Regla (Heatmap)
if greeting_results:
    score_cols = ['R1_Score', 'R2_Score', 'R3_Score', 'R4_Score', 'R5_Score']
    
    fig = go.Figure(data=go.Heatmap(
        z=df_greeting[score_cols].values,
        x=['R1: Validación', 'R2: Empatía', 'R3: Ofertas', 'R4: Respeto', 'R5: Cierre'],
        y=df_greeting['Audio'],
        colorscale='RdYlGn',
        text=df_greeting[score_cols].values,
        texttemplate='%{text:.0f}',
        textfont={"size": 10},
        colorbar=dict(title="Score")
    ))
    
    fig.update_layout(
        title='Scores por Regla (Evaluación LLM)',
        xaxis_title='Regla',
        yaxis_title='Audio',
        height=max(400, len(df_greeting) * 25)
    )
    
    fig.show()

## 6. 💾 Exportar Resultados

In [ ]:
# Exportar a CSV
if greeting_results:
    report_path = config.OUTPUT_DIR / "reporte_calidad_llm.csv"
    df_greeting.to_csv(report_path, index=False, encoding='utf-8-sig')
    print(f"✅ Reporte exportado a: {report_path}")
    
    # Mostrar tabla
    display(df_greeting)

In [ ]:
import pandas as pd
import plotly.express as px

# 1. Extraer datos para comparación
comparison_data = []
for config_name, results in all_results.items():
    for res in results:
        comparison_data.append({
            "Config": config_name,
            "Audio": res['audio_file'][:30], # Truncar para visualización
            "Score": res['global_score'],
            "Compliance": res['compliance']['global_result']
        })

if comparison_data:
    df_comp = pd.DataFrame(comparison_data)

    # 2. Gráfico de Barras Comparativo
    fig = px.bar(df_comp, x="Audio", y="Score", color="Config", barmode="group",
                 title="Comparación de Scores Globales por Configuración de LLM",
                 labels={"Score": "Puntuación (0-100)", "Audio": "Archivo de Audio", "Config": "Configuración"})
    fig.show()

    # 3. Resumen Promedio por Configuración
    avg_scores = df_comp.groupby("Config")["Score"].mean().reset_index()
    fig_avg = px.pie(avg_scores, values="Score", names="Config", 
                     title="Distribución de Calidad Promedio por Configuración")
    fig_avg.show()

    print("Resumen Comparativo:")
    display(avg_scores)
else:
    print("⚠️ No hay datos para comparar")